> **DO YOU USE GITHUB?**  
If True: print('Remember to make your edits in a personal copy of this notebook')  
Else: print('You don't have to understand. Continue your life.')

# Module 7: Web Scraping 2

In module_6 your learned some powerful tricks. Tricks that will work when the data is already shipped in a neat format. However this is often not the case. In this session we shall learn the art of parsing unstructured text and a more principled and advanced method of parsing HTML.

This will help you build ***custom datasets*** within just a few hours or days work, that would have taken ***months*** to curate and clean manually.



Readings for `module 6+7+8`:
- [Python for Data Analysis, chapter 6](https://bedford-computing.co.uk/learning/wp-content/uploads/2015/10/Python-for-Data-Analysis.pdf)
- [A Practical Introduction to Web Scraping in Python](https://realpython.com/python-web-scraping-practical-introduction/)
- [An introduction to web scraping with Python](https://towardsdatascience.com/an-introduction-to-web-scraping-with-python-a2601e8619e5)
- [Introduction to Web Scraping using Selenium](https://medium.com/the-andela-way/introduction-to-web-scraping-using-selenium-7ec377a8cf72)

Video materiale from `ISDS 2020`:
- [Web Scraping 1](https://bit.ly/ISDS2021_6)
- [Web Scraping 2](https://bit.ly/ISDS2021_7)
- [Web Scraping 3](https://bit.ly/ISDS2021_8)

Other ressources:
- [Nicklas Webpage](https://nicklasjohansen.netlify.app/)
- [Data Driven Organizational Analysis, Fall 2021](https://efteruddannelse.kurser.ku.dk/course/2021-2022/ASTK18379U)
- [Master of Science (MSc) in Social Data Science](https://www.socialdatascience.dk/education)


## Introduction to HTML
[What is HTML?](https://www.w3schools.com/whatis/whatis_html.asp)  

HTML has a Tree structure. 

Each node in the tree has:
- Children, siblings, parents, descendants. 
- Ids and attributes

<img src="http://www.openbookproject.net/tutorials/getdown/css/images/lesson4/HTMLDOMTree.png"/>


## Important syntax and patterns
_______________
```html 
<p>The p tag indicates a paragraph <p/>
```
_______________
```html 
<b>The b tag makes the text bold, giving us a clue to its importance </b>
```
output: <b>The b tag makes the text bold, giving us a clue to its importance </b>
```html 

<em>The em tag emphasize the text</em>, giving us a clue to its importance
```
output: <em>The em tag makes emphasize the text</em>, giving us a clue to its importance
___________
```html 
<h1>h1</h1><h2>h2</h2><h3>h3</h3><b>Headers give similar clues</b>
```
output:
<h1>h1</h1><h2>h2</h2><h3>h3</h3><b>Headers give similar clues</b>  
  
```html 
<a href="www.google.com">The a tag creates a hyperlink <a/>
```
output: <a href="www.google.com">The a tag creates a hyperlink <a/>

## How do we find our way around this tree?
1. Regex: Extracting string patterns using .split and regular expresssions
2. CSS-selectors: Specifying paths using css-selectors, xpath syntax.
3. ```BeautifulSoup```: A more powerful, principled and readable way to parse data and navigate HTML

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import selenium
import time
import pandas as pd

### Regex
- [What is regex?](https://en.wikipedia.org/wiki/Regular_expression)
- The brute force way is to parse by convering your downloded matriale into a large string
- Now you can create standard string operations
- And apply smart regex to identify the data you are looking for e.g. links.

In [2]:
url = 'https://www.theguardian.com/us-news/2019/aug/14/taco-eating-contest-death-fresno-california'
response = requests.get(url)
html = response.text
#html.split('\n')
#re.findall("(?P<url>https?://[^\s]+)", html)[0]

### CSS Selectors 
- [What is a CSS Selector?](https://www.w3schools.com/css/css_selectors.asp)
- Another way to browse through the HTML tree
- Define a unique path to an element in the HTML tree.
- It is quick but has to be hardcoded and also more likely to break.
- [Nicklas recommend using this free Google Chrome CSS Selector](https://chrome.google.com/webstore/detail/selectorgadget/mhjhnkcfbdhnjickkkdbjoemdmbfginb)

In [3]:
url = 'https://www.theguardian.com/us-news/2019/aug/14/taco-eating-contest-death-fresno-california'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
soup.select('.dcr-125vfar')[0].text

'Man dies after taco-eating contest in California'

## Parsing HTML with BeautifulSoup
BeautifulSoup makes the html tree navigable. 
It allows you to:
    * Search for elements by tag name and/or by attribute.
    * Iterate through them, go up, sideways or down the tree.
    * Furthermore it helps you with standard tasks such as extracting raw text from html,
    which would be a very tedious task if you had to hardcode it using `.split` commands and using your own regular expressions will be unstable.

In [4]:
url = 'https://www.theguardian.com/us-news/2019/aug/14/taco-eating-contest-death-fresno-california'
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html,'lxml') # parse the raw html using BeautifoulSoup

In [5]:
# example: finding hyperlinks
links = soup.find_all('a') # find all a tags -connoting a hyperlink.
[link['href'] for link in links if link.has_attr('href')][0:5] # unpack the hyperlink from the a nodes.

['#maincontent',
 '#navigation',
 '/preference/edition/int',
 '/preference/edition/uk',
 '/preference/edition/us']

In [6]:
# example: finding headline
headline = soup.find('h1') # search for the first headline: h1 tag. 
name = headline['class'][0].strip() # use the class attribute name as column name.
value = headline.text.strip() # extract text using build in method.
print(name,':',value)

dcr-125vfar : Man dies after taco-eating contest in California


In [7]:
# example: finding article_text
article_text = soup.findAll('div', {'class':'dcr-185kcx9'})[0]
article_text.text

'A man died shortly after competing in a taco-eating contest at a minor league baseball game in California, authorities said Wednesday.Dana Hutchings, 41, of Fresno, died Tuesday night shortly after arriving at a hospital, said Tony Botti, a Fresno sheriff spokesman.An autopsy on Hutchings will be done Thursday to determine a cause of death, Botti said. It was not immediately known how many tacos the man had eaten or whether he had won the contest.Paul Braverman, a spokesman for the Fresno Grizzlies, said in a statement that the team was “devastated to learn” of the fan’s death and that the team would “work closely with local authorities and provide any helpful information that is requested”.Tuesday night’s competition came before Saturday’s World Taco Eating Championship, to be held at Fresno’s annual Taco Truck Throwdown. The team on Wednesday announced that it was canceling that contest.Matthew Boylan, who watched Tuesday’s taco eating contest from his seat in the stadium, told the 

Say we are interested in how articles cite sources to back up their story i.e. their hyperlink behaviour within the article, and we want to see if the media has changed their behaviour.

We know how to search for links. But the cool part is that we can search from anywhere in the HTML tree. This means that once we have located the article content node - as above - we can search from there. This results in hyperlinks used within the article text.


In [8]:
# example: finding citation links
citations = article_text.findAll('a')

citation_links = [] # define container to the hyperlinks
for citation in citations: # iterate through each citation node
    if citation.has_attr('data-link-name'): # check if it has the right attribute
        if citation['data-link-name'] =='in body link': # and if the value of that attribute is correct
            print(citation['href'])
            citation_links.append(citation['href']) #  add link to the container

citation_links

https://www.theguardian.com/us-news/california
https://www.theguardian.com/us-news/gallery/2016/jul/04/nathans-famous-hotdog-eating-contest-in-pictures


['https://www.theguardian.com/us-news/california',
 'https://www.theguardian.com/us-news/gallery/2016/jul/04/nathans-famous-hotdog-eating-contest-in-pictures']

## Creating a dataset from www.bold.dk

### Let's put together some of the stuff we have learned so far
1. **Investigate:** In this example we will try to investigate the website to uderstand its structure. 
2. **Mapping:** Then we will try to collect all the urls and save them into a list
3. **Parsing:** At last, we will try to collect the information in each url in a simpel loop.

#### First, we pay around with the site trying to understand its structure

In [9]:
# define our URL
url = 'https://www.bold.dk/' 

# connects to site
response = requests.get(url)

# parse data with BeautifulSoup
soup = BeautifulSoup(response.text,'lxml') # parse the HTML

#identify articles to scrape by inspecting site
articles = soup.find_all('div',{'class':'news_list_item'}) # search for the ul node

# checking if articles match website
for i in range(1):
    print(articles[i].text.strip())

# identifying how to find an url from an article
article_url = articles[0].attrs['data-vr-contentbox-url']
print(article_url)

Officielt: Tottenham skriver med Romero

21:18
https://www.bold.dk/fodbold/nyheder/officielt-tottenham-skriver-med-romero/


In [10]:
articles[0]

<div article_id="389341" class="news_list_item" data-vr-contentbox="#1" data-vr-contentbox-url="https://www.bold.dk/fodbold/nyheder/officielt-tottenham-skriver-med-romero/" id="news_list_item_389341" position="1" tag_ids="10549,8190">
<div class="checkbox ball no_text_select" style="margin:7px 6px 3px 2px">
<img class="unchecked" src="https://s3.eu-central-1.amazonaws.com/static.bold.dk/img/sprites/newslist_checkbox_20x40.png"/>
</div>
<a class="title" href="/fodbold/nyheder/officielt-tottenham-skriver-med-romero/"><span data-vr-headline="">Officielt: Tottenham skriver med Romero</span>
<img src="https://s3.eu-central-1.amazonaws.com/static.bold.dk/img/tag/180x180/8206.png" style="position: absolute;width: 12px;right: 28px;top: 7px;"/>
<span class="font9 note-grey" style="right:2px;">21:18</span>
</a>
</div>

#### Second, we create a list of urls that we want to scrape

In [11]:
url = 'https://www.bold.dk/' 
response = requests.get(url)
soup = BeautifulSoup(response.text,'lxml')
articles = soup.find_all('div',{'class':'news_list_item'})

# create an empty list
list_of_article_urls = []

# creating a loop that appends the article url to the list above
for i in range(len(articles)):
    list_of_article_urls.append(articles[i].attrs['data-vr-contentbox-url'])

#printing the list
#list_of_article_urls

#printing one example
print(list_of_article_urls[0])

https://www.bold.dk/fodbold/nyheder/officielt-tottenham-skriver-med-romero/


#### Third, we scrape each site from the url list

In [12]:
# this step usually reuqiere a new step of investigation
# to figure out what information you want to download
# in this example we want the title, the lead and time posted

# creatig empty list for the infomation we want to extract for every article
h1_list = []
lead = []
time_posted = []

for i in range(10): # 10 #len(list_of_article_urls)
    
    # this time we scrape for each news article in the url list we created before
    url = list_of_article_urls[i]
    response = requests.get(url)
    soup = BeautifulSoup(response.text,'lxml')
    
    # pedagogical way of append title to list
    temp_1 = soup.find_all('h1')
    temp_1 = temp_1[1]
    temp_1 = temp_1.text.strip()
    h1_list.append(temp_1)
    
    # how I would actually do it
    lead.append(soup.find_all('div',{'class':'lead'})[0].text.strip())
    
    # sometimes you make wierd things - that works
    temp_3 = soup.find_all('time')
    temp_3 = temp_3[0]
    temp_3 = str(temp_3)[16:32]
    time_posted.append(temp_3)

In [13]:
# h1 
soup.find_all('h1')

[<h1 class="break_new_headline"></h1>,
 <h1>Medie: Rostov dropper Andreas Maxsø</h1>,
 <h1 class="title">Fodbold - Seneste nyheder</h1>]

In [14]:
soup.find_all('h1')[1]

<h1>Medie: Rostov dropper Andreas Maxsø</h1>

In [15]:
soup.find_all('h1')[1].text.strip()

'Medie: Rostov dropper Andreas Maxsø'

In [16]:
# lead
soup.find_all('div',{'class':'lead'})[0].text.strip()

'Det russiske medie Eurostavka skriver, at FK Rostov har droppet at hente danske Andreas Maxsø, da klubbens nye cheftræner har foretrukket Bastos til forsvaret.'

In [17]:
# time_posted
soup.find_all('time')[0]

<time datetime="2021-08-06 18:49">06.08.2021 18:49</time>

#### Lastly, we put our collected information into a dataframe

In [18]:
import pandas as pd
df = pd.DataFrame({'title':h1_list, 'lead':lead, 'time':time_posted})
df

,title,lead,time
0,Officielt: Tottenham skriver med Romero,Tidligere fredag købte Atalanta 23-årige Crist...,2021-08-06 21:18
1,LIVE: FC Midtjylland foran 1-0 ved pausen,"FC Midtjylland er ramt af adskillige afbud, nå...",2021-08-06 21:17
2,McCowatt sørgede for smal Helsingør-sejr,På et mål af Callum McCowatt i anden halvleg s...,2021-08-06 20:54
3,Hvidovre tildelte Fremad Amager nyt nederlag,På en hæsblæsende første halvleg fik Hvidovre ...,2021-08-06 20:36
4,Silkeborg tog sikker sejr i oprykkerduellen,"Sebastian Jørgensen, Nicolai Vallys og Alexand...",2021-08-06 20:25
5,SLUT: Silkeborg - Viborg minut for minut,Silkeborg har spillet 0-0 i alle Superliga-kam...,2021-08-06 20:23
6,Ansu Fati forventes klar til kamp i september,FC Barcelonas Ansu Fati er på vej tilbage efte...,2021-08-06 19:59
7,FCM skifter tre - Vejle uden ændringer,Bo Henriksen har fundet plads til blandt andre...,2021-08-06 19:34
8,West Bromwich sender kant til Saudi-Arabien,Championship-klubben West Bromwich har solgt b...,2021-08-06 19:11
9,Medie: Rostov dropper Andreas Maxsø,"Det russiske medie Eurostavka skriver, at FK R...",2021-08-06 18:49


In [19]:
# saving df
df.to_csv('df_bold.dk.csv')

# loading df
pd.read_csv('df_bold.dk.csv', index_col=0)

,title,lead,time
0,Officielt: Tottenham skriver med Romero,Tidligere fredag købte Atalanta 23-årige Crist...,2021-08-06 21:18
1,LIVE: FC Midtjylland foran 1-0 ved pausen,"FC Midtjylland er ramt af adskillige afbud, nå...",2021-08-06 21:17
2,McCowatt sørgede for smal Helsingør-sejr,På et mål af Callum McCowatt i anden halvleg s...,2021-08-06 20:54
3,Hvidovre tildelte Fremad Amager nyt nederlag,På en hæsblæsende første halvleg fik Hvidovre ...,2021-08-06 20:36
4,Silkeborg tog sikker sejr i oprykkerduellen,"Sebastian Jørgensen, Nicolai Vallys og Alexand...",2021-08-06 20:25
5,SLUT: Silkeborg - Viborg minut for minut,Silkeborg har spillet 0-0 i alle Superliga-kam...,2021-08-06 20:23
6,Ansu Fati forventes klar til kamp i september,FC Barcelonas Ansu Fati er på vej tilbage efte...,2021-08-06 19:59
7,FCM skifter tre - Vejle uden ændringer,Bo Henriksen har fundet plads til blandt andre...,2021-08-06 19:34
8,West Bromwich sender kant til Saudi-Arabien,Championship-klubben West Bromwich har solgt b...,2021-08-06 19:11
9,Medie: Rostov dropper Andreas Maxsø,"Det russiske medie Eurostavka skriver, at FK R...",2021-08-06 18:49


# Exercise Set 7: Web Scraping 2

In this Exercise Set we shall develop our webscraping skills even further by practicing **parsing** and navigating html trees using `BeautifoulSoup` and furthermore train extracting information from raw text with no html tags to help, using regular expressions. 

But just as importantly you will get a chance to think about **data quality issues** and how to ensure reliability when curating your own webdata. 

## Exercise Section 7.1: Logging and data quality

> **Ex. 7.1.1:** *`Why` is it important to log processes in your data collection?*



> **Ex. 7.1.2:**
*`How` does logging help with both ensuring and documenting the quality of your data?*


**Logging is important in order to keep track of the scraping to make sure that everything runs smooth. With a log we can check for any errors and locate them, which would make things a lot less tideous than trying to find the error manually. With scraping we are also responsible for the data that we collect and a log helps us document our job.**

## Exercise Section 7.2: Parsing a Table from HTML using BeautifulSoup.

In module_6 I showed you a neat little prepackaged function in pandas that did all the work. However today we should learn the mechanics of it. *(It is not just for educational purposes, sometimes the package will not do exactly as you want.)*

We hit the Basketball stats page from yesterday again: https://www.basketball-reference.com/leagues/NBA_2018.html.


> **Ex. 7.2.1:** Here we practice simply locating the table node of interest using the `find` method build into BeautifoulSoup. But first we have to fetch the HTML using the `requests` module. Parse the tree using `BeautifulSoup`. And then use the **>Inspector<** tool (* right click on the table < press inspect element *) in your browser to see how to locate the Eastern Conference table node - i.e. the *tag* name of the node, and maybe some defining *attributes*.

In [20]:
url = "https://www.basketball-reference.com/leagues/NBA_2018.html"
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, 'lxml')
EC_table = soup.find('table', {'id':'confs_standings_E'})



You have located the table should now build a function that starts at a "table node" and parses the information, and outputs a pandas DataFrame. 

Inspect the element either within the notebook or through the **>Inspector<** tool and start to see how a table is written in html. Which tag names can be used to locate rows? How will you iterate through columns. Were is the header located?

> **Ex. 7.2.2:** First you parse the header which can be found in the canonical tag name: thead. 
Next you use the `find_all` method to search for the tag, and iterate through each of the elements extracting the text, using the `.text` method builtin to the the node object. Store the header values in a list container. 

> **Ex. 7.2.3:** Next you locate the rows, using the canonical tag name: tbody. And from here you search for all rows tags. Fiugre out the tag name yourself, inspecting the tbody node in python or using the **Inspector**. 

> **Ex. 7.2.4:** Next run through all the rows and extract each value, similar to how you extracted the header. However here is a slight variation: Since each value node can have a different tag depending on whether it is a digit or a string, you should use the `.children` method instead of the `.find_all` - (or write compile a regex that matches both the td tag and the th tag.) 
>Once the value nodes of each row has been located using the `.children` method you should extract the value. Store the extracted rows as a list of lists: ```[[val1,val2,...valk],...]```

In [27]:
# 7.2.2

header_container = EC_table.find_all('thead')[0].text.split('\n')[2:-2]

# 7.2.3-4

row_container = EC_table.find('tbody').find_all('tr')

row_list = []

import numpy as np

def convert_value_type(value_node):
    if value_node.name == 'th':
        return value_node.text
    else: 
        try: 
            return float(value_node.text)
        except:
            return np.nan 

for row_node in row_container:
    row = []
    for child in row_node.children:
        row.append(convert_value_type(child))
    row_list.append(row)

df = pd.DataFrame(row_list, columns=header_container)

df.head()

,Eastern Conference,W,L,W/L%,GB,PS/G,PA/G,SRS
0,Toronto Raptors*,59.0,23.0,0.720,NaN,111.7,103.9,7.29
1,Boston Celtics*,55.0,27.0,0.671,4.0,104.0,100.4,3.23
2,Philadelphia 76ers*,52.0,30.0,0.634,7.0,109.8,105.3,4.30
3,Cleveland Cavaliers*,50.0,32.0,0.610,9.0,110.9,109.9,0.59
4,Indiana Pacers*,48.0,34.0,0.585,11.0,105.6,104.2,1.18


> **Ex. 7.2.5:** Now locate all tables from the page, using the `.find_all` method searching for the table tag name. Iterate through the table nodes and apply the function created for parsing html tables. Store each table in a dictionary using the table name as key. The name is found by accessing the id attribute of each table node, using dictionary-style syntax - i.e. `table_node['id']`.

In [33]:

def parse_html(table_node):

    header_container = table_node.find_all('thead')[0].text.split('\n')[2:-2]

    row_container = table_node.find('tbody').find_all('tr')

    row_list = []

    for row_node in row_container:
        row = []
        for child in row_node.children:
            row.append(convert_value_type(child))

        row_list.append(row)

    df = pd.DataFrame(row_list, columns=header_container)

    return df

df = parse_html(EC_table)
df.head()

,Eastern Conference,W,L,W/L%,GB,PS/G,PA/G,SRS
0,Toronto Raptors*,59.0,23.0,0.720,NaN,111.7,103.9,7.29
1,Boston Celtics*,55.0,27.0,0.671,4.0,104.0,100.4,3.23
2,Philadelphia 76ers*,52.0,30.0,0.634,7.0,109.8,105.3,4.30
3,Cleveland Cavaliers*,50.0,32.0,0.610,9.0,110.9,109.9,0.59
4,Indiana Pacers*,48.0,34.0,0.585,11.0,105.6,104.2,1.18


> **Ex. 7.2.6. (extra) :** Compare your results to the pandas implementation [pd.read_html](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_html.html)

## Exercise Section 7.3: The European Research Counsel (optional)
**NOTE** Exercise 7.3 is difficult and therefore also optional. I expect less than 10% of you being able to solve this one.

Imagine we wanted to analyze whether the European funding behaviour was biased towards certain countries and gender. We might decide to scrape who has received funding from the ERC.
https://erc.europa.eu/

* First we figure find navigate the grant listings.
* Next we figure out how to page these results. 
* And finally we want to grab the information.

### Data Storage and operating system interactions

> **Ex. 7.3.1:** *Import the python library `os`. Write pyhon code in this jupyter notebook creating a new folder in your directory called "erc_funding". Inside your new folder create 3 subfolders called 'mapping', 'raw_data' and 'parsed_data'.*



### Mapping

> **Ex. 7.3.2:** *Investigate [https://erc.europa.eu/projects-figures/erc-funded-projects/results?items_per_page=100&search_api_views_fulltext=&](https://erc.europa.eu/projects-figures/erc-funded-projects/results?items_per_page=100&search_api_views_fulltext=&). Figure out how many sites you need to loop thorugh. Save the response for each site using in `condecs` in your "mapping" subfolder. Use the `tqdm` to track your loop.
Use the Snorre Ralund Connector class to log your activity.*



### Parsing

> **Ex. 7.3.3:** *Write a function that takes a filename (from our mapping subfolder) as and input and returns (in our parsed_data subfolder) a `pandas`dataframe of parsed information. Use `os` library to navigate your operating system (paths) and `condecs` library to read files inside your function. Last, concatenate all your dataframes into one dataframe you call "df" consisting of all parsed data.*

### Reliability and Data Quality

##### Inspect the data
> **Ex. 7.3.4:** *Investigate your dataframe "df". Check for dublicates. Count NaN values. Create a `matplotlib` histrogram plot for every column of "df" illustrating the lenght of the string (x-axis) and row count( y-axis).*

##### Do simple descriptives
> **Ex. 7.3.5:** *Create a value_counts() for each of the three columns (Host Institution (HI), Researcher (PI) and Project acronym) in your "df". What can counting do for us in this exercise in terms of Reliability and Data Quality?*

##### Visualize the Log
> **Ex. 7.3.6:** *Load your "erc_log.csv". Convert the time column 't' to datetime. Use `matplotlib` to create three plots: (1) time it took to make the call, (2) the response size over time, and (3) the delta_t against the response_size .)*